<a href="https://colab.research.google.com/github/dffesalbon/bert-lexicon-sentiment-analysis/blob/lexicon-based/lexicon-based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from textblob.classifiers import DecisionTreeClassifier
from textblob.classifiers import MaxEntClassifier

import re, string

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
from collections import defaultdict
import nltk
nltk.download('punkt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
train = pd.read_csv('/content/sample_data/train.csv')
test = pd.read_csv('/content/sample_data/test.csv')

#cl = NaiveBayesClassifier(train)
class_names = ['Non-toxic', 'Toxic']

In [ ]:
def clean_text(text):
    text = re.sub('#', '', text) # Removing '#' hashtag
    text = re.sub('\w+:\/\/\S+', '', text) # Removing hyperlink
    text = re.sub('[^a-zA-Z]', ' ', text) # Remove punctuation
    text.lower()
    return text

In [ ]:
train.loc[:,'texts'] = train['texts'].apply(lambda x : clean_text(x).strip())
test.loc[:,'texts'] = test['texts'].apply(lambda x : clean_text(x).strip())

In [ ]:
train = train.drop(['match', 'slot'], 1)

<ipython-input-26-6a800c7913b0>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train = train.drop(['match', 'slot'], 1)


In [ ]:
#train

In [ ]:
#cl = NaiveBayesClassifier(train)
train_data = []
for index, row in train.iterrows():
    train_data.append((row['texts'], row['target']))
    #print(row['texts'], row['targe'])

In [ ]:
#print(train_data)
nb = NaiveBayesClassifier(train_data)
dt = DecisionTreeClassifier(train_data)
me = MaxEntClassifier(train_data)


In [ ]:
nbpred = []
dtpred = []
mepred = []
for index, row in test.iterrows():
    nbpred.append(int(nb.classify(row['texts'])))
    dtpred.append(int(dt.classify(row['texts'])))
    mepred.append(int(me.classify(row['texts'])))

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.501


/usr/local/lib/python3.8/dist-packages/nltk/classify/maxent.py:1381: RuntimeWarning: overflow encountered in power
  exp_nf_delta = 2 ** nf_delta
/usr/local/lib/python3.8/dist-packages/nltk/classify/maxent.py:1383: RuntimeWarning: invalid value encountered in multiply
  sum1 = numpy.sum(exp_nf_delta * A, axis=0)
/usr/local/lib/python3.8/dist-packages/nltk/classify/maxent.py:1384: RuntimeWarning: invalid value encountered in multiply
  sum2 = numpy.sum(nf_exp_nf_delta * A, axis=0)


         Final               nan        0.499


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
nb_predictions = pd.Series(nbpred)
dt_predictions = pd.Series(dtpred)
me_predictions = pd.Series(mepred)

#NaiveBayesClassifier

In [ ]:
print(classification_report(test['target'], nb_predictions, target_names=class_names))

              precision    recall  f1-score   support

   Non-toxic       0.71      0.89      0.79       270
       Toxic       0.85      0.63      0.73       267

    accuracy                           0.76       537
   macro avg       0.78      0.76      0.76       537
weighted avg       0.78      0.76      0.76       537



#DecisionTreeClassifier

In [ ]:
print(classification_report(test['target'], dt_predictions, target_names=class_names))

              precision    recall  f1-score   support

   Non-toxic       0.75      0.87      0.80       270
       Toxic       0.84      0.70      0.76       267

    accuracy                           0.79       537
   macro avg       0.79      0.79      0.78       537
weighted avg       0.79      0.79      0.78       537



#MaxEntClassifier

In [ ]:
print(classification_report(test['target'], me_predictions, target_names=class_names))

              precision    recall  f1-score   support

   Non-toxic       0.50      1.00      0.67       270
       Toxic       0.00      0.00      0.00       267

    accuracy                           0.50       537
   macro avg       0.25      0.50      0.33       537
weighted avg       0.25      0.50      0.34       537



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
